In [113]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.VERSION)
print(tf.keras.__version__)

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

1.11.0
2.1.6-tf


## Normal model

In [114]:
print(x_train.shape)
print(x_test.shape)
print(y_train[0:3])
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=99)
print(x_train.shape)
print(x_valid.shape)
print(y_train[0:3])

(60000, 28, 28)
(10000, 28, 28)
[5 0 4]
(48000, 28, 28)
(12000, 28, 28)
[4 4 4]


In [115]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [116]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [117]:
%%time 
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
fit = model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 6s 132us/step - loss: 0.2410 - acc: 0.9293 - val_loss: 0.1268 - val_acc: 0.9625
Epoch 2/5
48000/48000 [==============================] - 8s 158us/step - loss: 0.1062 - acc: 0.9675 - val_loss: 0.1016 - val_acc: 0.9685
Epoch 3/5
48000/48000 [==============================] - 7s 155us/step - loss: 0.0765 - acc: 0.9760 - val_loss: 0.0856 - val_acc: 0.9735
Epoch 4/5
48000/48000 [==============================] - 8s 158us/step - loss: 0.0587 - acc: 0.9815 - val_loss: 0.0698 - val_acc: 0.9803
Epoch 5/5
48000/48000 [==============================] - 8s 156us/step - loss: 0.0448 - acc: 0.9855 - val_loss: 0.0728 - val_acc: 0.9786
Wall time: 36.7 s


In [30]:
%%time 
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train_one_hot, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 17s 277us/step - loss: 0.0207 - acc: 0.9934
Epoch 2/5
60000/60000 [==============================] - 20s 336us/step - loss: 0.0158 - acc: 0.9946
Epoch 3/5
60000/60000 [==============================] - 20s 327us/step - loss: 0.0163 - acc: 0.9948
Epoch 4/5
60000/60000 [==============================] - 18s 292us/step - loss: 0.0157 - acc: 0.9946
Epoch 5/5
60000/60000 [==============================] - 22s 370us/step - loss: 0.0159 - acc: 0.9947


In [118]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape)
print(x_test.shape)
model = tf.keras.models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28, 28,1)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


(48000, 28, 28, 1)
(10000, 28, 28, 1)


In [57]:
%%time 
y_train_one_hot = tf.keras.utils.to_categorical(y_train, 10)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train_one_hot, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 194s 3ms/step - loss: 0.1162 - acc: 0.9637
Epoch 2/5
60000/60000 [==============================] - 186s 3ms/step - loss: 0.0388 - acc: 0.9881
Epoch 3/5
60000/60000 [==============================] - 187s 3ms/step - loss: 0.0239 - acc: 0.9925
Epoch 4/5
60000/60000 [==============================] - 180s 3ms/step - loss: 0.0149 - acc: 0.9948
Epoch 5/5
60000/60000 [==============================] - 184s 3ms/step - loss: 0.0104 - acc: 0.9967


# Define model
All layers must be defined in __init__() and run order in call()

<mark>TODO</mark>: what is compute_output_shape() -> change output size if nescessary. See **Define layer > Flatten layer** for more information

"[In case your layer modifies the shape of its input, you should specify here the shape transformation logic](https://keras.io/layers/writing-your-own-keras-layers/). This allows Keras to do automatic shape inference." (from keras page) - **mean, not necessary ?**

In [119]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.flatten_1= layers.Flatten(input_shape=(28, 28))
        self.dense_1 = layers.Dense(512, activation='relu')
        self.dropout_1 = layers.Dropout(0.2)
        self.dense_2 = layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.flatten_1(inputs)
        x = self.dense_1(x)
        x = self.dropout_1(x)
        return self.dense_2(x)
    
#     def compute_output_shape(self, input_shape):
#         shape = tf.TensorShape(input_shape).as_list()
#         shape[-1] = self.num_classes
#         return tf.TensorShape(shape)

In [120]:
model = MyModel(num_classes=10)
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
48000/48000 [==============================] - 9s 180us/step - loss: 0.3486 - acc: 0.8940
Epoch 2/5
48000/48000 [==============================] - 6s 123us/step - loss: 0.1184 - acc: 0.9653
Epoch 3/5
48000/48000 [==============================] - 6s 123us/step - loss: 0.0896 - acc: 0.9750
Epoch 4/5
48000/48000 [==============================] - 6s 129us/step - loss: 0.0727 - acc: 0.9795
Epoch 5/5
48000/48000 [==============================] - 6s 128us/step - loss: 0.0623 - acc: 0.9825


# Define layer
Define a layer is more more more difficult than Model

Some notes on this:
* Some tensorflow function takes a tensor of int (double, float ..) as argument. This means we cannot pass None ( number of sample ) to it. (See MyFlattenLayer)
* Always define compute_output_shape

In [172]:
import numpy as np
class MyFlattenLayer(layers.Layer):
    def __init__(self, data_format=None, **kwargs):
        self.data_format = data_format
        super(MyFlattenLayer, self).__init__(**kwargs)
        
        
    def call(self, inputs):
        if self.data_format=="channels_first":
            transpose_order = [0]
            transpose_order.extend([i for i in range(2, len(inputs.get_shape()))])
            transpose_order.append(1)
            inputs = tf.transpose(inputs, perm=transpose_order)
        output_shape = self.compute_output_shape(inputs.get_shape())
#         NOTE : Error occurred with following code, because input size is None, not a tensor of int32
#         outputs = tf.reshape(inputs, output_shape)
        outputs = tf.reshape(inputs, (tf.shape(inputs)[0], output_shape[1]))
        return outputs
    
    def compute_output_shape(self, input_shape):
        input_shape = tf.TensorShape(input_shape).as_list()
        output_shape = [input_shape[0]]
        if all(input_shape[1:]):
            output_shape += [np.prod(input_shape[1:])]
        else :
            output_shape += [None]
        return tf.TensorShape(output_shape)
    
    def get_config(self):
        base_config = super(MyFlattenLayer, self).get_config()
        base_config['data_format'] = self.data_format
        return base_config
   

In [175]:
model = tf.keras.Sequential([
    MyFlattenLayer(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_valid, y_valid))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 58us/step - loss: 0.3838 - acc: 0.8851 - val_loss: 0.1666 - val_acc: 0.9512
Epoch 2/5
48000/48000 [==============================] - 2s 52us/step - loss: 0.1319 - acc: 0.9612 - val_loss: 0.1310 - val_acc: 0.9616
Epoch 3/5
48000/48000 [==============================] - 3s 53us/step - loss: 0.0951 - acc: 0.9724 - val_loss: 0.1075 - val_acc: 0.9681
Epoch 4/5
48000/48000 [==============================] - 4s 92us/step - loss: 0.0746 - acc: 0.9779 - val_loss: 0.1002 - val_acc: 0.9711
Epoch 5/5
48000/48000 [==============================] - 2s 52us/step - loss: 0.0613 - acc: 0.9819 - val_loss: 0.0935 - val_acc: 0.9742


In [247]:
class MyDenseLayer(layers.Layer):
    def __init__(self, output_dim, activation, **kwargs):
        self.output_dim = output_dim
        self.activation = activation
        super(MyDenseLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim)).as_list()
        self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='uniform', 
                                      dtype=tf.float32,
                                  trainable=True)
        super(MyDenseLayer, self).build(input_shape)
        
    def call(self, inputs):
        y = layers.Activation(self.activation)(tf.matmul(inputs, self.kernel))
        return y

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
    def get_config(self):
        base_config = super(MyDenseLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        base_config['activation'] = self.activation
        return base_config

In [249]:
model = tf.keras.Sequential([
    MyFlattenLayer(),
    MyDenseLayer(128, 'relu'),
    MyDenseLayer(10, 'softmax')
])

model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_valid, y_valid))

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 8s 157us/step - loss: 0.4660 - acc: 0.8647 - val_loss: 0.1984 - val_acc: 0.9421
Epoch 2/5
48000/48000 [==============================] - 3s 69us/step - loss: 0.1606 - acc: 0.9539 - val_loss: 0.1389 - val_acc: 0.9597
Epoch 3/5
48000/48000 [==============================] - 4s 90us/step - loss: 0.1145 - acc: 0.9669 - val_loss: 0.1181 - val_acc: 0.9658
Epoch 4/5
48000/48000 [==============================] - 4s 90us/step - loss: 0.0908 - acc: 0.9735 - val_loss: 0.1087 - val_acc: 0.9697
Epoch 5/5
48000/48000 [==============================] - 5s 112us/step - loss: 0.0741 - acc: 0.9784 - val_loss: 0.1087 - val_acc: 0.9706
